<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Mw-Pradeep/Bioco_Mw_Superveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [2]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.3 MB/s eta 0:00:00


Importering av nødvendig bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np
import copy
import math
from statistics import mean, stdev

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline

In [4]:
# Modeller
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [5]:
# Hyperoptimalisering
import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

# Egendefinerte moduler

In [6]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_trening_validering_testing_evaluering.py /content/moduler_trening_validering_testing_evaluering.py

/content


In [7]:
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_enzym
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_dag
from moduler_trening_validering_testing_evaluering import trening_validering_oppdeling_dag

from moduler_trening_validering_testing_evaluering import enzymfordeling
from moduler_trening_validering_testing_evaluering import n_splitt_enzym
from moduler_trening_validering_testing_evaluering import Resultat_evaluering
from moduler_trening_validering_testing_evaluering import res_eval_opt
from moduler_trening_validering_testing_evaluering import res_eval
from moduler_trening_validering_testing_evaluering import Resultat_dicts

In [8]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_veiledet.py /content/moduler_optimalisering_evaluering_veiledet.py

/content


In [9]:
from moduler_optimalisering_evaluering_veiledet import trening_rfr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_knr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_svr_enzym

from moduler_optimalisering_evaluering_veiledet import trening_rfr_dag
from moduler_optimalisering_evaluering_veiledet import trening_knr_dag
from moduler_optimalisering_evaluering_veiledet import trening_svr_dag

from moduler_optimalisering_evaluering_veiledet import veiledet_modell_evaluering_1
from moduler_optimalisering_evaluering_veiledet import veiledet_modell_evaluering_2

# Importering av relevant data

In [10]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Design produksjon
data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data_design.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [11]:
#Tilfeldighetsfrø
random_seed = 2024

# Lagring av resultater

In [12]:
# Oppretter instans for lagring av resultater
res = Resultat_dicts()

# Setter fast testandel på 25%

In [13]:
# Fast testsandel
test_andel = 0.25

# Treningsandeler til undersøkelse
treningsandeler = [0.25, 0.5, 0.75]

# Parametere til hyperoptimalisering

In [18]:
# Antall runder til optimalisering

# Antall runder til hel treningsandel
n_runder_hel = 75

# Antall runder til mindre treningsandeler
n_runder_andel = 50

# Parametervalg for kryssvalidering
n_gjentagelser = 5

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: Hele treningssett

In [15]:
# Setter treningsandel
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data,
                                   test_andel=test_andel)

In [16]:
# Finner antall splitt etter minste antall av enzymtype
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_hel_opt,
                             dict_res=res.dict_res_1_hel,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder_hel,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

[I 2024-04-28 13:47:27,115] A new study created in memory with name: no-name-28cb2341-fe08-4c3b-b4d9-7eede28d05b3
[I 2024-04-28 13:47:28,781] Trial 0 finished with value: 553.7280596844948 and parameters: {'n_neighbors': 6, 'weights': 'distance', 'p': 2}. Best is trial 0 with value: 553.7280596844948.
[I 2024-04-28 13:47:28,990] Trial 1 finished with value: 635.524422364756 and parameters: {'n_neighbors': 1, 'weights': 'distance', 'p': 8}. Best is trial 0 with value: 553.7280596844948.
[I 2024-04-28 13:47:30,575] Trial 3 finished with value: 556.4791424804832 and parameters: {'n_neighbors': 8, 'weights': 'distance', 'p': 2}. Best is trial 0 with value: 553.7280596844948.
[I 2024-04-28 13:47:30,732] Trial 2 finished with value: 558.1694219304126 and parameters: {'n_neighbors': 6, 'weights': 'distance', 'p': 4}. Best is trial 0 with value: 553.7280596844948.
[I 2024-04-28 13:47:32,622] Trial 4 finished with value: 571.6733845837996 and parameters: {'n_neighbors': 8, 'weights': 'distance'

Lagring av resultater

In [17]:
res_1_hel = pd.DataFrame(res.dict_res_1_hel)
res_1_hel_opt = pd.DataFrame(res.dict_res_1_hel_opt)

res_1_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_hel.csv')
res_1_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_hel_opt.csv')

Velger beste modell

## Alternativ 1: Forskjellige treningsandeler

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, _, _, _ = trening_testsett_oppdeling_enzym(rå_data,
                                                       trening,
                                                       test_andel=ubenyttet_andel,
                                                       random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert

  # Finner antall splitt etter minste antall av enzymtype
  n_splitt = n_splitt_enzym(rå_data,
                            treningssett_markert)

  # Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
  veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_hel_opt,
                               dict_res=res.dict_res_1_hel,
                               treningssett_markert=treningssett_markert,
                               enzymtyper_treningssett_markert=enzymtyper_treningssett_markert,
                               testsett=testsett,
                               n_splitt=n_splitt,
                               n_gjentagelser=n_gjentagelser,
                               n_runder=n_runder_andel,
                               test_andel=test_andel,
                               trenings_andel=trenings_andel,
                               random_seed=random_seed)

[I 2024-04-28 16:48:46,918] A new study created in memory with name: no-name-9e6e02b0-8cb1-4cf1-a9a5-08ddf209c942
[I 2024-04-28 16:48:47,306] Trial 0 finished with value: 824.9691330819086 and parameters: {'n_neighbors': 4, 'weights': 'distance', 'p': 1}. Best is trial 0 with value: 824.9691330819086.
[I 2024-04-28 16:48:47,331] Trial 1 finished with value: 890.2088466532051 and parameters: {'n_neighbors': 1, 'weights': 'distance', 'p': 6}. Best is trial 0 with value: 824.9691330819086.
[I 2024-04-28 16:48:47,717] Trial 2 finished with value: 819.4545194203469 and parameters: {'n_neighbors': 2, 'weights': 'uniform', 'p': 5}. Best is trial 2 with value: 819.4545194203469.
[I 2024-04-28 16:48:47,720] Trial 3 finished with value: 897.0927500076665 and parameters: {'n_neighbors': 6, 'weights': 'distance', 'p': 2}. Best is trial 2 with value: 819.4545194203469.
[I 2024-04-28 16:48:48,074] Trial 4 finished with value: 1323.3556225626214 and parameters: {'n_neighbors': 10, 'weights': 'uniform

Lagring av resultater

In [ ]:
res_1_hel = pd.DataFrame(res.dict_res_1_hel)
res_1_hel_opt = pd.DataFrame(res.dict_res_1_hel_opt)

res_1_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_hel.csv')
res_1_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_hel_opt.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Alternativ 2: Hele treningssett

In [ ]:
# Definerer treningsandel
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Enzymtyper Fordeling: Test
            Frekvens  Andel
EnzymeCode                 
A2                25  27.78
B                 16  17.78
C                 15  16.67
E                 15  16.67
D                 13  14.44
A1                 6   6.67


In [ ]:
# Finner antall splitt etter minste antall av enzymtype
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_hel_opt,
                             dict_res=res.dict_res_2_hel,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder_hel,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Lagring av resultater

In [ ]:
res_2_hel = pd.DataFrame(res.dict_res_2_hel)
res_2_hel_opt = pd.DataFrame(res.dict_res_2_hel_opt)

res_2_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_hel.csv')
res_2_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_hel_opt.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Velger beste modell

## Alternativ 2: Forskjellige treningssandeler

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, _, _\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       test_ekskluder_kalkun_kylling=True)

  else:
    treningssett_markert = trening_markert

  #Enzymfordeling
  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)
  print(enzym_treningssett_markert)

  # n_splitt
  n_splitt = n_splitt_enzym(rå_data, treningssett_markert)

  veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_hel_opt,
                               dict_res=res.dict_res_2_hel,
                               rå_data=rå_data,
                               treningssett_markert=treningssett_markert,
                               testsett=testsett,
                               n_splitt=n_splitt,
                               n_runder=n_runder_andel,
                               test_andel=test_andel,
                               trenings_andel=trenings_andel,
                               random_seed=random_seed)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Enzymtyper Fordeling: Treningssett markert andel: 0.25
            Frekvens  Andel
EnzymeCode                 
A2                46  28.22
C                 30  18.40
D                 30  18.40
B                 28  17.18
E                 26  15.95
A1                 3   1.84
Enzymtyper Fordeling: Treningssett markert andel: 0.5
            Frekvens  Andel
EnzymeCode                 
A2                73  35.96
C                 32  15.76
D                 32  15.76
E                 30  14.78
B                 29  14.29
A1                 7   3.45
Enzymtyper Fordeling: Treningssett markert andel: 0.75
            Frekvens  Andel
EnzymeCode                 
A2               103  37.45
E                 45  16.36
C                 40  14.55
B                 39  14.18
D                 37  13.45
A1                11   4.00


Lagring av resultater

In [ ]:
res_2_hel = pd.DataFrame(res.dict_res_2_hel)
res_2_hel_opt = pd.DataFrame(res.dict_res_2_hel_opt)

res_2_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_hel.csv')
res_2_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_hel_opt.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Designproduksjon

## Alternativ 1: Hele treningssett

In [ ]:
# Definere treningsandel
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design,
                                   test_andel=test_andel)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Finner antall splitt etter minste antall av enzymtype
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_design_opt,
                             dict_res=res.dict_res_1_design,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder_hel,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Lagring av resultater

In [ ]:
res_1_design = pd.DataFrame(res.dict_res_1_design)
res_1_design_opt = pd.DataFrame(res.dict_res_1_design_opt)

res_1_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_design.csv')
res_1_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_design_opt.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Velger beste modell

## Alternativ 1: Forskjellige treningsandeler

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, _, _, _ = trening_testsett_oppdeling_enzym(rå_data,
                                                       trening,
                                                       test_andel=ubenyttet_andel,
                                                       random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert

  # n_splitt
  n_splitt = n_splitt_enzym(rå_data, treningssett_markert)

  # Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
  veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_design_opt,
                               dict_res=res.dict_res_1_design,
                               treningssett_markert=treningssett_markert,
                               enzymtyper_treningssett_markert=enzymtyper_treningssett_markert,
                               testsett=testsett,
                               n_splitt=n_splitt,
                               n_gjentagelser=n_gjentagelser,
                               n_runder=n_runder_andel,
                               test_andel=test_andel,
                               trenings_andel=trenings_andel,
                               random_seed=random_seed)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Lagring av resultater

In [ ]:
res_1_design = pd.DataFrame(res.dict_res_1_design)
res_1_design_opt = pd.DataFrame(res.dict_res_1_design_opt)

res_1_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_design.csv')
res_1_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_design_opt.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Alternativ 2: Hele treningssett

In [ ]:
# Definerer treningsandel
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

Enzymtyper Fordeling: Test
            Frekvens  Andel
EnzymeCode                 
E                 15  22.73
B                 13  19.70
C                 13  19.70
D                 12  18.18
A2                 9  13.64
A1                 4   6.06


In [ ]:
# Finner antall splitt etter minste antall av enzymtype
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_design_opt,
                             dict_res=res.dict_res_2_design,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder_hel,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

## Alternativ 2: Forskjellige treningssandeler

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, _, _\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       test_ekskluder_kalkun_kylling=True)

  else:
    treningssett_markert = trening_markert

  #Enzymfordeling
  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)
  print(enzym_treningssett_markert)

  # n_splitt
  n_splitt = n_splitt_enzym(rå_data, treningssett_markert)

  veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_design_opt,
                               dict_res=res.dict_res_2_design,
                               rå_data=rå_data,
                               treningssett_markert=treningssett_markert,
                               testsett=testsett,
                               n_splitt=n_splitt,
                               n_runder=n_runder_andel,
                               test_andel=test_andel,
                               trenings_andel=trenings_andel,
                               random_seed=random_seed)

Enzymtyper Fordeling: Treningssett markert andel: 0.2
            Frekvens  Andel
EnzymeCode                 
C                 30  22.06
D                 30  22.06
B                 27  19.85
E                 26  19.12
A2                21  15.44
A1                 2   1.47
Enzymtyper Fordeling: Treningssett markert andel: 0.4
            Frekvens  Andel
EnzymeCode                 
C                 31  21.09
D                 31  21.09
B                 29  19.73
E                 28  19.05
A2                22  14.97
A1                 6   4.08
Enzymtyper Fordeling: Treningssett markert andel: 0.6
            Frekvens  Andel
EnzymeCode                 
E                 36  21.95
C                 33  20.12
D                 32  19.51
B                 31  18.90
A2                24  14.63
A1                 8   4.88
Enzymtyper Fordeling: Treningssett markert andel: 0.8
            Frekvens  Andel
EnzymeCode                 
E                 48  22.54
B                 42  19.72


Lagring av resultater

In [ ]:
res_2_design = pd.DataFrame(res.dict_res_2_design)
res_2_design_opt = pd.DataFrame(res.dict_res_2_design_opt)

res_2_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_design.csv')
res_2_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_design_opt.csv')

# NIR målinger ekskludert

In [ ]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

## Hele datasett

## Alternativ 1: Hele treningssett

In [ ]:
# Setter treningsandel
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_uten_NIR,
                                   test_andel=test_andel)

# Finner antall splitt etter minste antall av enzymtype
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_hel_opt_NIR,
                             dict_res=res.dict_res_1_hel_NIR,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder_hel,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

Resultater

In [ ]:
res_1_hel_NIR = pd.DataFrame(res.dict_res_1_hel_NIR)
res_1_hel_opt_NIR = pd.DataFrame(res.dict_res_1_hel_opt_NIR)

res_1_hel_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_hel_NIR.csv')
res_1_hel_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_hel_opt_NIR.csv')

## Alternativ 2: Hele treningssett

In [ ]:
# Definerer treningandel
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_uten_NIR,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

# Finner antall splitt etter minste antall av enzymtype
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_hel_opt_NIR,
                             dict_res=res.dict_res_2_hel_NIR,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder_hel,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

Resultater

In [ ]:
res_2_hel_NIR = pd.DataFrame(res.dict_res_2_hel_NIR)
res_2_hel_opt_NIR = pd.DataFrame(res.dict_res_2_hel_opt_NIR)

res_2_hel_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_hel_NIR.csv')
res_2_hel_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_hel_opt_NIR.csv')

## Designproduksjon

## Alternativ 1: Hele treningssett

In [ ]:
# Setter treningsandel
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design_uten_NIR,
                                   test_andel=test_andel)

# Finner antall splitt etter minste antall av enzymtype
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_design_opt_NIR,
                             dict_res=res.dict_res_1_design_NIR,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder_hel,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

Resultater

In [ ]:
res_1_design_NIR = pd.DataFrame(res.dict_res_1_design_NIR)
res_1_design_opt_NIR = pd.DataFrame(res.dict_res_1_design_opt_NIR)

res_1_design_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_design_NIR.csv')
res_1_design_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_design_opt_NIR.csv')

## Alternativ 2: Hele treningssett

In [ ]:
# Definerer treningsandel
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

# Finner antall splitt etter minste antall av enzymtype
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_design_opt_NIR,
                             dict_res=res.dict_res_2_design_NIR,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder_hel,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

Resultater

In [ ]:
res_2_design_NIR = pd.DataFrame(res.dict_res_2_design_NIR)
res_2_design_opt_NIR = pd.DataFrame(res.dict_res_2_design_opt_NIR)

res_2_design_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_design_NIR.csv')
res_2_design_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_design_opt_NIR.csv')